# Implementing methods

Once we have a mock server, we could already provide an interface for
external services to mock our replies. This is very helpful to enable
clients to test our API and enable quick feedbacks on data types and possible responses.


Now that we have the contract, we should start with the implementation!


## OperationId

OperationId is the OAS3 fields with maps the resource-target with the python function to call.

```
paths:
  /status
    get:
      ...
      operationId: api.get_status
      ...
```

The method signature should reflect the function's one.

OAS allows to pass parameters to the resource target via:
- query parameters
- http headers
- request body

### Implement get_status

At first we'll just implement the `get_status` in [api.py](/edit/notebooks/oas3/api.py) function that:

- takes no input parameters;
- returns a problem+json



In [1]:
# connexion provides a predefined problem object
from connexion import problem

    
# Exercise: write a get_status() returning a successful response to problem.
help(problem)

Help on function problem in module connexion.problem:

problem(status, title, detail, type=None, instance=None, headers=None, ext=None)
    Returns a `Problem Details <https://tools.ietf.org/html/draft-ietf-appsawg-http-problem-00>`_ error response.
    
    
    :param status: The HTTP status code generated by the origin server for this occurrence of the problem.
    :type status: int
    :param title: A short, human-readable summary of the problem type.  It SHOULD NOT change from occurrence to
                  occurrence of the problem, except for purposes of localisation.
    :type title: str
    :param detail: An human readable explanation specific to this occurrence of the problem.
    :type detail: str
    :param type: An absolute URI that identifies the problem type.  When dereferenced, it SHOULD provide human-readable
                 documentation for the problem type (e.g., using HTML).  When this member is not present its value is
                 assumed to be "about:blank

In [2]:
def get_status():
    return problem(
        status=200,
        title="OK",
        detail="The application is working properly"
    )
    

Now  [run the spec in a terminal](/terminals/1) using

```
cd /code/notebooks/oas3/
connexion run /code/notebooks/oas3/ex-03-02-path.yaml
```

and play a bit with the [Swagger UI](https://TODO)

## Defining endpoints in OAS3

Now that we have added our metadata, we can provide the informations about endpoints.
OAS3 allows multiple endpoint because every good API has a test / sandbox infrastructure.

```
# One or more server
#   You can add production, staging and test environments.
#   We are using the custom `x-sandbox` to identify
#   sandbox instances
servers:
  - description: Development server
    url: https://localhost:8443/datetime/v1
  - description: Test server
    url: https://api.example.com/datetime/v1
    x-sandbox: true
    # The following custom parameter can be used
    # to declare how to check the API.
    x-healthCheck:
      url: https://api.example.com/datetime/v1/status
      interval: 300
      timeout: 15

```

### Exercise

Edit the `servers` attribute so that it points to your actual endpoint URL (eg. your IP/port).

## paths

Now we can define our first `path` that is the `/status` one.
An interoperable API should have a well known url usable for testing its status, so that
the implementers can decide which is a suitable method for testing it (eg. it could be
a simple OK/KO method or can execute basic checks (eg. databases are reachable, smoke testing other components, ..)

NB: the `/status` path is not a replacement for proper monitoring your APIs, but a way to communicate to your peers that you're online.

An OAS3 path references:
    
- the associated METHOD
- a `summary` and a `description` of the method


```  
  /status:
    get:
      summary: Ritorna lo stato dell'applicazione.
      description: |
        Ritorna lo stato dell'applicazione. A scopo
        di test, su base randomica puo' ritornare
        un errore.
```
- a reference to the python object to call when the 
   
```
      operationId: get_status
```

- the http statuses of the possible responses, each with its description,
  content-type and examples
      
```
      responses:
        '200':
          description: |
            Il server ha ritornato lo status. In caso di problemi
            ritorna sempre un problem+json.
          content:
            application/problem+json:
              example:
                status: 200
                title: OK
                detail: API is working properly.
        default:
          description: |
            If none of the above statuses is returned, then this applies
          content:
            application/problem+json:
              example:
                status: 500
                title: Internal Server Error
                detail: API is not responding correctly
            
           

```

### Exercise

We haven't already implemented the `operationId`
`get_status()`, so  [to run the spec in a terminal](/terminals/1) we should tell the server
to ignore this.

```
connexion run /code/notebooks/oas3/ex-03-02-path.yaml --stub
```

### Exercise

1- What happens if I get the `/status` resource of my API now?

2- And if I invoke another path which is not mentioned in the spec?

3- Restart the server via

```
connexion run /code/notebooks/oas3/ex-03-02-path.yaml --mock notimplemented
```

   What happens now when invoking ```curl http://0.0.0.0:8889/datetime/v1/status``` ?

Solution on the unimplemented method

```
$ curl http://0.0.0.0:8889/datetime/v1/status
{
  "detail": "Empty module name",
  "status": 501,
  "title": "Not Implemented",
  "type": "about:blank"
}
```

Solution on other paths

```
$ curl http://0.0.0.0:8889/datetime/v1/missing
{
  "detail": "The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.",
  "status": 404,
  "title": "Not Found",
  "type": "about:blank"
}
```

## Schemas

OAS3 allows defining, using and reusing schemas. They can be defined inline,  in the `component` section or referenced from another file, like below.

```
components:
    schemas:
      Problem:
        $ref: 'https://teamdigitale.github.io/openapi/0.0.5/definitions.yaml#/schemas/Problem'

```


In [ ]:
# Let's download the Problem schema
from requests import get
ret  = get('https://teamdigitale.github.io/openapi/0.0.5/definitions.yaml')

# Yaml parse the definitions
definitions = yaml.safe_load(ret.content)

# Nicely print the Problem schema
print(yaml.dump(definitions['schemas']['Problem']))

In [1]:
### Exercise
# Use this cell to list all the structures present in definitions

In [ ]:
for sections, v in definitions.items():
    for items, vv in v.items():
        print(f'{sections}.{items}')

## Exercise

Edit [ex-03-02-path.yaml](/edit/notebooks/oas3/ex-03-02-path.yaml) so that every `/status` response uses
the `Problem` schema.

Look at [simple.yaml](https://github.com/teamdigitale/api-starter-kit/blob/master/openapi/simple.yaml) to
see a complete implementation.

In [ ]:
## Exercise

Test the new setup

